In [2]:
path = ('C://Users//Jongjae//Project/home_credit/')
trainset = pd.read_csv(path + 'application_trainset.csv')
testset = pd.read_csv(path + 'application_testset.csv')
# pre_app_train = pd.read_csv(path + 'previous_application_train.csv')
# pos_cash_train = pd.read_csv(path + 'POS_CASH_balance_train.csv')
# install_train = pd.read_csv(path + 'installments_payments_train.csv')
# credit_card_train = pd.read_csv(path + 'credit_card_balance_train.csv')
# bureau_train = pd.read_csv(path + 'bureau_train.csv')
# bureau_balance_train = pd.read_csv(path + 'bureau_balance_train.csv')

In [4]:
del trainset['Unnamed: 0']
del testset['Unnamed: 0']
# del bureau_train['Unnamed: 0']
# # del bureau_train['Unnamed: 0.1']
# del bureau_balance_train['Unnamed: 0']
# # del bureau_balance_train['Unnamed: 0.1']
# del credit_card_train['Unnamed: 0']
# del install_train['Unnamed: 0']
# del pre_app_train['Unnamed: 0']
# # del pre_app_train['Unnamed: 0.1']
# del pos_cash_train['Unnamed: 0']

In [3]:
print(trainset.shape)

(184506, 122)


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

## Application_trainset.csv PreProcessing

## 1 ~ 5 Columns 
- 1) SK_ID_CURR 
    - `ID 값`.
    - Missing Values 없음.
    - **전처리 X**
- 2) TARGET 
    - `Target 값`.
    - `0 또는 1` 
    - Missing Values 없음.
    - **전처리 X**
- 3) NAME_CONTRACT_TYPE
    - Text이기에, 숫자 형태로 변환 필요함. (Encoding 필요)
        - `Cash loans` - `0`
        - `Revolving loans` - `1` 
- 4) CODE_GENDER
    - `F`: 121,353개
        - `F` => `0`
    - `M`: 63,150개
        - `M` => `1`
    - `XNA`: 4개 
        - `XNA` => `2`
- 5) FLAG_OWN_CAR
    - `Y`: 62,645개
        - `Y` => `1`
    - `N`: 121,861개
        - `N` => `0`

In [5]:
trainset['NAME_CONTRACT_TYPE'] = LabelEncoder().fit_transform(trainset['NAME_CONTRACT_TYPE'])
trainset['CODE_GENDER'] = LabelEncoder().fit_transform(trainset['CODE_GENDER'])
trainset['FLAG_OWN_CAR'] = LabelEncoder().fit_transform(trainset['FLAG_OWN_CAR'])

## 6 ~ 10 Columns 
- 6) FLAG_OWN_REALTY
    - 전처리 필요함
        - `Y`: 128,057개
            - `Y` => `1`
        - `N`: 56,449개
            - `N` => `0`
    - Missing Values 없음.
- 7) CNT_CHILDRN
    - 전처리 필요함 (Category)
    - 0 ~ 19명 (총 14명)
        - `0` => `0`
        - `1` => `1`
        - `2` => `2`
        - `3` => `3`
        - `4` => `4`
        - `5` => `5`
        - `6` => `6`
        - `7` => `7`
        - `8` => `8`
        - `10` => `9`
        - `11` => `10`
        - `12` => `11`
        - `14` => `12`
        - `19` => `13`
            = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 8) AMT_INCOME_TOTAL
    - Scaling 필요함 
    - Numeric 변수이기에, scaling 적용
    - Missing Values 없음.
- 9) AMT_CREDIT
    - Scaling 필요함 
    - Numeric 변수이기에, scaling 적용
    - Missing Values 없음.
- 10) AMT_ANNUITY
    - Scaling 필요함 
    - Numeric 변수이기에, scaling 적용
    - Missing Values 7개 있음
        - Missing values = > mean 값 적용 후, 스케일링 가능

In [6]:
trainset['FLAG_OWN_REALTY'] = LabelEncoder().fit_transform(trainset['FLAG_OWN_REALTY'])

In [7]:
from sklearn.preprocessing import LabelBinarizer
trainset_CNT_CHILDREN = pd.DataFrame(LabelBinarizer().fit_transform(trainset['CNT_CHILDREN']), 
                                           columns = ['CNT_CHILDREN_0', 'CNT_CHILDREN_1', 'CNT_CHILDREN_2',
                                                    'CNT_CHILDREN_3', 'CNT_CHILDREN_4', 'CNT_CHILDREN_5',
                                                    'CNT_CHILDREN_6', 'CNT_CHILDREN_7', 'CNT_CHILDREN_8',
                                                    'CNT_CHILDREN_10', 'CNT_CHILDREN_11', 'CNT_CHILDREN_12',
                                                    'CNT_CHILDREN_14', 'CNT_CHILDREN_19'], index=trainset.index)
trainset = pd.concat([trainset, trainset_CNT_CHILDREN], axis = 1)
del(trainset['CNT_CHILDREN'])

In [8]:
trainset['AMT_INCOME_TOTAL'] = scale(trainset['AMT_INCOME_TOTAL'])
trainset['AMT_CREDIT'] = scale(trainset['AMT_CREDIT'])

In [9]:
trainset['AMT_ANNUITY'].fillna(trainset['AMT_ANNUITY'].mean(), inplace = True)
trainset['AMT_ANNUITY'] = scale(trainset['AMT_ANNUITY'])

## 11 ~ 15 Columns 
- 11) AMT_GOODS_PRICE
    - Scaling 필요함 
    - Missing Values 278 values. = > mean 값으로 대체
- 12) NAME_TYPE_SUITE
    - 전처리 필요함 (Binary로 변환)
    - ['Unaccompanied', 'Family', 'Spouse, partner', 'Children',
       'Other_A', nan, 'Other_B', 'Group of people'],
            = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 1292 values. => mode 값으로 대체
- 13) NAME_INCOME_TYPE
    - 전처리 필요함 (Binary로 변환) 
    - ['Working', 'State servant', 'Commercial associate', 'Pensioner',
       'Unemployed', 'Student', 'Businessman', 'Maternity leave']
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 14) NAME_EDUCATION_TYPE
    - 전처리 필요함 (Binary로 변환) 
    - ['Secondary / secondary special', 'Higher education',
       'Incomplete higher', 'Lower secondary', 'Academic degree']
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 15) NAME_FAMILY_STATUS
    - 전처리 필요함 (Binary로 변환) 
    - ['Single / not married', 'Married', 'Civil marriage', 'Widow',
       'Separated', 'Unknown']
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨

In [10]:
trainset['AMT_GOODS_PRICE'].fillna(trainset['AMT_GOODS_PRICE'].mean(), inplace = True)
trainset['AMT_GOODS_PRICE'] = scale(trainset['AMT_GOODS_PRICE'])

In [11]:
trainset['NAME_TYPE_SUITE'].fillna(trainset['NAME_TYPE_SUITE'].mode()[0], inplace = True)
trainset_NAME_TYPE_SUITE = pd.get_dummies(trainset['NAME_TYPE_SUITE'])
trainset = pd.concat([trainset, trainset_NAME_TYPE_SUITE], axis = 1)
del(trainset['NAME_TYPE_SUITE'])

In [12]:
trainset_NAME_INCOME_TYPE = pd.get_dummies(trainset['NAME_INCOME_TYPE'])
trainset = pd.concat([trainset, trainset_NAME_INCOME_TYPE], axis = 1)
del(trainset['NAME_INCOME_TYPE'])

In [13]:
trainset_NAME_EDUCATION_TYPE = pd.get_dummies(trainset['NAME_EDUCATION_TYPE'])
trainset = pd.concat([trainset, trainset_NAME_EDUCATION_TYPE], axis = 1)
del(trainset['NAME_EDUCATION_TYPE'])

In [14]:
trainset_NAME_FAMILY_STATUS = pd.get_dummies(trainset['NAME_FAMILY_STATUS'])
trainset = pd.concat([trainset, trainset_NAME_FAMILY_STATUS], axis = 1)
del(trainset['NAME_FAMILY_STATUS'])

## 16 ~ 20 Columns 
- 16) NAME_HOUSING_TYPE
    - 전처리 필요함 (Binary로 변환)
    - Missing Values 없음
    - ['House / apartment', 'Rented apartment', 'With parents',
       'Municipal apartment', 'Office apartment', 'Co-op apartment']
            = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
            
- 17) REGION_POPULATION_RELATIVE
    - 이미 정규화 된 값 (전처리 필요 X) 
    - Missing Values 없음
    
- 18) DAYS_BIRTH
    - Numeric으로 보고, 전처리 Scaling 필요함.
    - Missing Values 없음.
- 19) DAYS_EMPLOYED
    - Numeric으로 보고, 전처리 Scaling 필요함.
    - Missing Values 없음.
- 20) DAYS_REGISTRATION
    - Numeric으로 보고, 전처리 Scaling 필요함.
    - Missing Values 없음.

In [15]:
trainset_NAME_HOUSING_TYPE = pd.get_dummies(trainset['NAME_HOUSING_TYPE'])
trainset = pd.concat([trainset, trainset_NAME_HOUSING_TYPE], axis = 1)
del(trainset['NAME_HOUSING_TYPE'])

In [16]:
trainset['DAYS_BIRTH'] = scale(trainset['DAYS_BIRTH'])
trainset['DAYS_EMPLOYED'] = scale(trainset['DAYS_EMPLOYED'])
trainset['DAYS_REGISTRATION'] = scale(trainset['DAYS_REGISTRATION'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


## 21 ~ 25 Columns 
- 21) DAYS_ID_PUBLISH
    - Numeric으로 보고, 전처리 Scaling 필요함.
    - Missing Values 없음.
- 22) **OWN_CAR_AGE** (상의가 필요해 보임)
    - 일단, OWN_CAR_AGE는 Numeric 형태로 간주. 0 ~ 91년식 포함 in one column
    - Missing Values 202,929 values
- 23) FLAG_MOBILE
    - 0 또는 1로 Category 값 으로 변경
    - Missing Values 없음.
- 24) FLAG_EMP_PHONE
    - 0 또는 1로 Category 값 으로 변경
    - Missing Values 없음.
- 25) FLAG_WORK_PHONE
    - 0 또는 1로 Category 값 으로 변경
    - Missing Values 없음.

In [17]:
trainset['DAYS_ID_PUBLISH'] = scale(trainset['DAYS_ID_PUBLISH'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [18]:
trainset['OWN_CAR_AGE'].fillna(trainset['OWN_CAR_AGE'].mode()[0], inplace=True)

In [19]:
print(sorted(trainset['OWN_CAR_AGE'].unique()))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 49.0, 50.0, 51.0, 52.0, 54.0, 55.0, 63.0, 64.0, 65.0]


In [20]:
from sklearn.preprocessing import LabelBinarizer
trainset_OWN_CAR_AGE = pd.DataFrame(LabelBinarizer().fit_transform(trainset['OWN_CAR_AGE']), 
                                           columns = ['OWN_CAR_AGE_0', 'OWN_CAR_AGE_1', 'OWN_CAR_AGE_2',
                                                    'OWN_CAR_AGE_3', 'OWN_CAR_AGE_4', 'OWN_CAR_AGE_5',
                                                    'OWN_CAR_AGE_6', 'OWN_CAR_AGE_7', 'OWN_CAR_AGE_8',
                                                    'OWN_CAR_AGE_9', 'OWN_CAR_AGE_10', 'OWN_CAR_AGE_11',
                                                    'OWN_CAR_AGE_12', 'OWN_CAR_AGE_13', 'OWN_CAR_AGE_14',
                                                     'OWN_CAR_AGE_15', 'OWN_CAR_AGE_16', 'OWN_CAR_AGE_17',
                                                     'OWN_CAR_AGE_18', 'OWN_CAR_AGE_19', 'OWN_CAR_AGE_20', 
                                                     'OWN_CAR_AGE_21', 'OWN_CAR_AGE_22', 'OWN_CAR_AGE_23', 
                                                     'OWN_CAR_AGE_24', 'OWN_CAR_AGE_25', 'OWN_CAR_AGE_26',
                                                     'OWN_CAR_AGE_27', 'OWN_CAR_AGE_28', 'OWN_CAR_AGE_29', 
                                                     'OWN_CAR_AGE_30', 'OWN_CAR_AGE_31', 'OWN_CAR_AGE_32',
                                                     'OWN_CAR_AGE_33', 'OWN_CAR_AGE_34', 'OWN_CAR_AGE_35', 
                                                     'OWN_CAR_AGE_36', 'OWN_CAR_AGE_37', 'OWN_CAR_AGE_38', 
                                                     'OWN_CAR_AGE_39', 'OWN_CAR_AGE_40', 'OWN_CAR_AGE_41', 
                                                     'OWN_CAR_AGE_42', 'OWN_CAR_AGE_43', 'OWN_CAR_AGE_44', 
                                                     'OWN_CAR_AGE_45', 'OWN_CAR_AGE_46', 'OWN_CAR_AGE_49',
                                                     'OWN_CAR_AGE_50', 'OWN_CAR_AGE_51', 'OWN_CAR_AGE_52',
                                                     'OWN_CAR_AGE_54', 'OWN_CAR_AGE_55', 'OWN_CAR_AGE_63',
                                                     'OWN_CAR_AGE_64', 'OWN_CAR_AGE_65'], index=trainset.index)
trainset = pd.concat([trainset, trainset_OWN_CAR_AGE], axis = 1)
del(trainset['OWN_CAR_AGE'])

In [21]:
trainset['FLAG_MOBIL'] = LabelEncoder().fit_transform(trainset['FLAG_MOBIL'])
trainset['FLAG_EMP_PHONE'] = LabelEncoder().fit_transform(trainset['FLAG_EMP_PHONE'])
trainset['FLAG_WORK_PHONE'] = LabelEncoder().fit_transform(trainset['FLAG_WORK_PHONE'])

## 26 ~ 30 Columns 
- 26) FLAG_CONT_MOBILE
    - 0 또는 1로 Category 값 으로 변경
    - Missing Values 없음.
- 27) FLAG_PHONE
    - 0 또는 1로 Category 값 으로 변경
    - Missing Values 202,929 values
- 28) FLAG_EMAIL
    - 0 또는 1로 Category 값 으로 변경
    - Missing Values 없음.
- 29) OCCUPATION_TYPE
    - ['Laborers', 'Core staff', 'Accountants', 'Managers', nan,
       'Drivers', 'Sales staff', 'Cleaning staff', 'Cooking staff',
       'Private service staff', 'Medicine staff', 'Security staff',
       'High skill tech staff', 'Waiters/barmen staff',
       'Low-skill Laborers', 'Realty agents', 'Secretaries', 'IT staff',
       'HR staff']
            = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 96,391 values
- 30) CNT_FAM_MEMBERS
    - [ 1.,  2.,  3.,  4.,  5.,  6.,  9.,  7.,  8., 10., 13., 14.,
       12., 20., 15., 16., 11.]
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.

In [22]:
trainset['FLAG_CONT_MOBILE'] = LabelEncoder().fit_transform(trainset['FLAG_CONT_MOBILE'])

In [23]:
trainset['FLAG_PHONE'].fillna(trainset['FLAG_PHONE'].mode()[0], inplace=True)
trainset['FLAG_PHONE'] = LabelEncoder().fit_transform(trainset['FLAG_PHONE'])

In [24]:
trainset['OCCUPATION_TYPE'].fillna(trainset['OCCUPATION_TYPE'].mode()[0], inplace=True)
trainset_OCCUPATION_TYPE = pd.get_dummies(trainset['OCCUPATION_TYPE'])
trainset = pd.concat([trainset, trainset_OCCUPATION_TYPE], axis = 1)
del(trainset['OCCUPATION_TYPE'])

In [25]:
from sklearn.preprocessing import LabelBinarizer
trainset_CNT_FAM_MEMBERS = pd.DataFrame(LabelBinarizer().fit_transform(trainset['CNT_FAM_MEMBERS']), 
                                           columns = ['CNT_FAM_MEMBERS_1', 'CNT_FAM_MEMBERS_3', 'CNT_FAM_MEMBERS_2',
                                                    'CNT_FAM_MEMBERS_4', 'CNT_FAM_MEMBERS_5', 'CNT_FAM_MEMBERS_6',
                                                    'CNT_FAM_MEMBERS_7', 'CNT_FAM_MEMBERS_8', 'CNT_FAM_MEMBERS_10',
                                                    'CNT_FAM_MEMBERS_9', 'CNT_FAM_MEMBERS_14', 'CNT_FAM_MEMBERS_12',
                                                    'CNT_FAM_MEMBERS_20', 'CNT_FAM_MEMBERS_15', 'CNT_FAM_MEMBERS_16',
                                                     'CNT_FAM_MEMBERS_13'], index=trainset.index)
trainset = pd.concat([trainset, trainset_CNT_FAM_MEMBERS], axis = 1)
del(trainset['CNT_FAM_MEMBERS'])

## 31 ~ 35 Columns 
- 31) REGION_RATING_CLIENT
    - 1, 2, 3
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 32) REGION_RATING_CLIENT_W_CITY
    - 1, 2, 3
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 33) WEEKDAY_APPR_PROCESS_START
    - Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 34) HOUR_APPR_PROCESS_START
    - 0 ~ 24 시간
           = > 모두 Dummy Variables로 DataFrame 끝으로 이동 됨
    - Missing Values 없음.
- 35) REG_REGION_NOT_LIV_REGION
    - 0 또는 1
           = > 그냥 둔다
    - Missing Values 없음.

In [26]:
trainset_REGION_RATING_CLIENT = pd.DataFrame(LabelBinarizer().fit_transform(trainset['REGION_RATING_CLIENT']), 
                                           columns = ['REGION_RATING_CLIENT_1', 'REGION_RATING_CLIENT_2', 'REGION_RATING_CLIENT_3'], index=trainset.index)
trainset = pd.concat([trainset, trainset_REGION_RATING_CLIENT], axis = 1)
del(trainset['REGION_RATING_CLIENT'])

In [27]:
trainset_REGION_RATING_CLIENT_W_CITY = pd.DataFrame(LabelBinarizer().fit_transform(trainset['REGION_RATING_CLIENT_W_CITY']), 
                                           columns = ['REGION_RATING_CLIENT_W_CITY_1', 'REGION_RATING_CLIENT_W_CITY_2', 'REGION_RATING_CLIENT_W_CITY_3'], index=trainset.index)
trainset = pd.concat([trainset, trainset_REGION_RATING_CLIENT_W_CITY], axis = 1)
del(trainset['REGION_RATING_CLIENT_W_CITY'])

In [28]:
trainset_WEEKDAY_APPR_PROCESS_START = pd.get_dummies(trainset['WEEKDAY_APPR_PROCESS_START'])
trainset = pd.concat([trainset, trainset_WEEKDAY_APPR_PROCESS_START], axis = 1)
del(trainset['WEEKDAY_APPR_PROCESS_START'])

In [29]:
trainset_HOUR_APPR_PROCESS_START = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['HOUR_APPR_PROCESS_START']), 
                                           columns = ['HOUR_APPR_PROCESS_START_1', 'HOUR_APPR_PROCESS_START_2', 'HOUR_APPR_PROCESS_START_3',
                                                      'HOUR_APPR_PROCESS_START_4', 'HOUR_APPR_PROCESS_START_5', 'HOUR_APPR_PROCESS_START_6',
                                                      'HOUR_APPR_PROCESS_START_7', 'HOUR_APPR_PROCESS_START_8', 'HOUR_APPR_PROCESS_START_9',
                                                      'HOUR_APPR_PROCESS_START_9', 'HOUR_APPR_PROCESS_START_10', 'HOUR_APPR_PROCESS_START_11',
                                                     'HOUR_APPR_PROCESS_START_12', 'HOUR_APPR_PROCESS_START_13', 'HOUR_APPR_PROCESS_START_14',
                                                     'HOUR_APPR_PROCESS_START_15', 'HOUR_APPR_PROCESS_START_16', 'HOUR_APPR_PROCESS_START_17',
                                                     'HOUR_APPR_PROCESS_START_18', 'HOUR_APPR_PROCESS_START_19', 'HOUR_APPR_PROCESS_START_20',
                                                     'HOUR_APPR_PROCESS_START_21', 'HOUR_APPR_PROCESS_START_22', 'HOUR_APPR_PROCESS_START_23'], index=trainset.index)
trainset = pd.concat([trainset, trainset_HOUR_APPR_PROCESS_START], axis = 1)
del(trainset['HOUR_APPR_PROCESS_START'])

## 36 ~ 40 Columns 
- 36) REG_REGION_NOT_WORK_REGION
    - 0 or 1 => 그대로 둔다
- 37) LIVE_REGION_NOT_WORK_REGION
    - 0 or 1 => 그대로 둔다
- 38) REG_CITY_NOT_LIVE_CITY
    - 0 or 1 => 그대로 둔다
- 39) REG_REGION_NOT_WORK_REGION
    - 0 or 1 => 그대로 둔다
- 40) LIVE_REGION_NOT_WORK_REGION
    - 0 or 1 => 그대로 둔다

## 41 Column
- 41) ORGANIZATION_TYPE
    - 58개의 기관

In [30]:
trainset_ORGANIZATION_TYPE = pd.get_dummies(trainset['ORGANIZATION_TYPE'])
trainset = pd.concat([trainset, trainset_ORGANIZATION_TYPE], axis = 1)
del(trainset['ORGANIZATION_TYPE'])

## 42 ~ 91 정규화 된 변수 
- 42) EXT_SOURCE_1
- 43) EXT_SOURCE_2
- 44) EXT_SOURCE_3
- 45) APARTMENTS_AVG
- 46) BASEMENTSARE_AVG
- 47) YEARS_BEGINEXPLUATATION_AVG
- 48) YEARS_BUILD_AVG
- 49) COMMONAREA_AVG
- 50) ELEVATORS_AVG
- 51) ENTRANCES_AVG
- 52) FLOORSMAX_AVG
- 53) FLOORSMIN_AVG
- 54) LANDAREA_AVG
- 55) LIVINGAPARTMENTS_AVG
- 56) LIVINGAREA_AVG
- 57) NONLIVINGAPARTMENTS_AVG
- 58) NONLIVINGAREA_AVG
- 59) APARTMENTS_MODE
- 60) BASEMENTSAREA_MODE
- 61) YEARS_BEGINEXPLUATATION_MODE
- 62) YEARS_BUILD_MODE
- 63) COMMONAREA_MODE
- 64) ELEVATORS_MODE
- 65) ENTRANCES_MODE
- 66) FLOORSMAX_MODE
- 67) FLOORSMIN_MODE
- 68) LANDAREA_MODE
- 69) LIVINGAPARTMENTS_MODE
- 70) LIVINGAREA_MODE
- 71) NONLIVINGAPARTMENTS_MODE
- 72) NONLIVINGAREA_MODE
- 73) APARTMENTS_MEDI
- 74) BASEMENTSAREA_MEDI
- 75) YEARS_BEGINEXPLUATATION_MEDI
- 76) YEARS_BUILD_MEDI
- 77) COMMONAREA_MEDI
- 78) ELEVATORS_MEDI
- 79) ENTRANCES_MEDI
- 80) FLOORSMAX_MEDI
- 81) FLOORSMIN_MEDI
- 82) LANDAREA_MEDI
- 83) LIVINGAPARTMENTS_MEDI
- 84) LIVINGAREA_MEDI
- 85) NONLIVINGAPARTMENTS_MEDI
- 86) NONLIVINGAREA_MEDI
- 87) FONDKAPREMONT_MODE
- 88) HOUSETYPE_MODE
- 89) TOTALAREA_MODE
- 90) WALLSMATERIAL_MODE
- 91) EMERGENCYSTATE_MODE

## 92 ~ 96
- 92) OBS_30_CNT_SOCIAL_CIRCLE
    - Missing Value 처리
    - Dummies 처리
- 93) OBS_30_CNT_SOCIAL_CIRCLE
    - Missing Value 처리
    - Dummies 처리
- 94) OBS_30_CNT_SOCIAL_CIRCLE
    - Missing Value 처리
    - Dummies 처리
- 95) OBS_30_CNT_SOCIAL_CIRCLE
    - Missing Value 처리
    - Dummies 처리
- 96) DAYS_LAST_PHONE_CHANGE
    - Numeric으로 보고, 전처리 Scaling 필요함.
    - Missing Values 없음.

In [31]:
trainset['OBS_30_CNT_SOCIAL_CIRCLE'] = trainset['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(trainset['OBS_30_CNT_SOCIAL_CIRCLE'].mode()[0])

In [32]:
trainset['OBS_30_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [33]:
print(sorted(trainset['OBS_30_CNT_SOCIAL_CIRCLE'].unique()))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 29.0, 30.0]


In [34]:
from sklearn.preprocessing import LabelBinarizer

trainset_OBS_30_CNT_SOCIAL_CIRCLE = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['OBS_30_CNT_SOCIAL_CIRCLE']), 
                                           columns = ['OBS_30_CNT_SOCIAL_CIRCLE_0', 'OBS_30_CNT_SOCIAL_CIRCLE_1', 'OBS_30_CNT_SOCIAL_CIRCLE_2',
                                                      'OBS_30_CNT_SOCIAL_CIRCLE_3', 'OBS_30_CNT_SOCIAL_CIRCLE_4', 'OBS_30_CNT_SOCIAL_CIRCLE_5',
                                                      'OBS_30_CNT_SOCIAL_CIRCLE_6', 'OBS_30_CNT_SOCIAL_CIRCLE_7', 'OBS_30_CNT_SOCIAL_CIRCLE_8',
                                                      'OBS_30_CNT_SOCIAL_CIRCLE_9', 'OBS_30_CNT_SOCIAL_CIRCLE_10', 'OBS_30_CNT_SOCIAL_CIRCLE_11',
                                                     'OBS_30_CNT_SOCIAL_CIRCLE_12', 'OBS_30_CNT_SOCIAL_CIRCLE_13', 'OBS_30_CNT_SOCIAL_CIRCLE_14',
                                                     'OBS_30_CNT_SOCIAL_CIRCLE_15', 'OBS_30_CNT_SOCIAL_CIRCLE_16', 'OBS_30_CNT_SOCIAL_CIRCLE_17',
                                                     'OBS_30_CNT_SOCIAL_CIRCLE_18', 'OBS_30_CNT_SOCIAL_CIRCLE_19', 'OBS_30_CNT_SOCIAL_CIRCLE_20',
                                                     'OBS_30_CNT_SOCIAL_CIRCLE_21', 'OBS_30_CNT_SOCIAL_CIRCLE_22', 'OBS_30_CNT_SOCIAL_CIRCLE_23',
                                                      'OBS_30_CNT_SOCIAL_CIRCLE_24', 'OBS_30_CNT_SOCIAL_CIRCLE_25', 'OBS_30_CNT_SOCIAL_CIRCLE_26',
                                                     'OBS_30_CNT_SOCIAL_CIRCLE_27', 'OBS_30_CNT_SOCIAL_CIRCLE_29', 'OBS_30_CNT_SOCIAL_CIRCLE_30'], index=trainset.index)
trainset = pd.concat([trainset, trainset_OBS_30_CNT_SOCIAL_CIRCLE], axis = 1)
del(trainset['OBS_30_CNT_SOCIAL_CIRCLE'])

In [35]:
trainset['DEF_30_CNT_SOCIAL_CIRCLE'].isnull().sum()

594

In [36]:
sorted(trainset['DEF_30_CNT_SOCIAL_CIRCLE'].unique())

[0.0, 1.0, 2.0, nan, 3.0, 4.0, 5.0, 6.0, 8.0]

In [37]:
trainset['DEF_30_CNT_SOCIAL_CIRCLE'].mode()

0    0.0
dtype: float64

In [38]:
trainset['DEF_30_CNT_SOCIAL_CIRCLE'] = trainset['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0.0)

In [39]:
trainset['DEF_30_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [40]:
from sklearn.preprocessing import LabelBinarizer

trainset_DEF_30_CNT_SOCIAL_CIRCLE = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['DEF_30_CNT_SOCIAL_CIRCLE']), 
                                           columns = ['DEF_30_CNT_SOCIAL_CIRCLE_0', 'DEF_30_CNT_SOCIAL_CIRCLE_1', 'DEF_30_CNT_SOCIAL_CIRCLE_2',
                                                      'DEF_30_CNT_SOCIAL_CIRCLE_3', 'DEF_30_CNT_SOCIAL_CIRCLE_4', 'DEF_30_CNT_SOCIAL_CIRCLE_5',
                                                      'DEF_30_CNT_SOCIAL_CIRCLE_6', 'DEF_30_CNT_SOCIAL_CIRCLE_8'], index=trainset.index)
trainset = pd.concat([trainset, trainset_DEF_30_CNT_SOCIAL_CIRCLE], axis = 1)
del(trainset['DEF_30_CNT_SOCIAL_CIRCLE'])

In [41]:
trainset['OBS_60_CNT_SOCIAL_CIRCLE'].isnull().sum()

594

In [42]:
print(sorted(trainset['OBS_60_CNT_SOCIAL_CIRCLE'].unique()))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, nan, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 29.0, 30.0]


In [43]:
trainset['OBS_60_CNT_SOCIAL_CIRCLE'].mode()

0    0.0
dtype: float64

In [44]:
trainset['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)

In [45]:
trainset['OBS_60_CNT_SOCIAL_CIRCLE'] = trainset['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0.0)

In [46]:
from sklearn.preprocessing import LabelBinarizer

trainset_OBS_60_CNT_SOCIAL_CIRCLE = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['OBS_60_CNT_SOCIAL_CIRCLE']), 
                                           columns = ['OBS_60_CNT_SOCIAL_CIRCLE_0', 'OBS_60_CNT_SOCIAL_CIRCLE_1', 'OBS_60_CNT_SOCIAL_CIRCLE_2',
                                                      'OBS_60_CNT_SOCIAL_CIRCLE_3', 'OBS_60_CNT_SOCIAL_CIRCLE_4', 'OBS_60_CNT_SOCIAL_CIRCLE_5',
                                                      'OBS_60_CNT_SOCIAL_CIRCLE_6', 'OBS_60_CNT_SOCIAL_CIRCLE_7', 'OBS_60_CNT_SOCIAL_CIRCLE_8',
                                                      'OBS_60_CNT_SOCIAL_CIRCLE_9', 'OBS_60_CNT_SOCIAL_CIRCLE_10', 'OBS_60_CNT_SOCIAL_CIRCLE_11',
                                                     'OBS_60_CNT_SOCIAL_CIRCLE_12', 'OBS_60_CNT_SOCIAL_CIRCLE_13', 'OBS_60_CNT_SOCIAL_CIRCLE_14',
                                                     'OBS_60_CNT_SOCIAL_CIRCLE_15', 'OBS_60_CNT_SOCIAL_CIRCLE_16', 'OBS_60_CNT_SOCIAL_CIRCLE_17',
                                                     'OBS_60_CNT_SOCIAL_CIRCLE_18', 'OBS_60_CNT_SOCIAL_CIRCLE_19', 'OBS_60_CNT_SOCIAL_CIRCLE_20',
                                                     'OBS_60_CNT_SOCIAL_CIRCLE_21', 'OBS_60_CNT_SOCIAL_CIRCLE_22', 'OBS_60_CNT_SOCIAL_CIRCLE_23',
                                                      'OBS_60_CNT_SOCIAL_CIRCLE_24', 'OBS_60_CNT_SOCIAL_CIRCLE_25', 'OBS_60_CNT_SOCIAL_CIRCLE_26',
                                                     'OBS_60_CNT_SOCIAL_CIRCLE_27', 'OBS_60_CNT_SOCIAL_CIRCLE_29', 'OBS_60_CNT_SOCIAL_CIRCLE_30'], index=trainset.index)
trainset = pd.concat([trainset, trainset_OBS_60_CNT_SOCIAL_CIRCLE], axis = 1)
del(trainset['OBS_60_CNT_SOCIAL_CIRCLE'])

In [47]:
trainset['DEF_60_CNT_SOCIAL_CIRCLE'].isnull().sum()

594

In [48]:
sorted(trainset['DEF_60_CNT_SOCIAL_CIRCLE'].unique())

[0.0, 1.0, 2.0, nan, 3.0, 4.0, 5.0, 6.0]

In [49]:
trainset['DEF_60_CNT_SOCIAL_CIRCLE'].mode()

0    0.0
dtype: float64

In [50]:
trainset['DEF_60_CNT_SOCIAL_CIRCLE'] = trainset['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0.0)

In [51]:
trainset['DEF_60_CNT_SOCIAL_CIRCLE'].isnull().sum()

0

In [52]:
from sklearn.preprocessing import LabelBinarizer

trainset_DEF_60_CNT_SOCIAL_CIRCLE = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['DEF_60_CNT_SOCIAL_CIRCLE']), 
                                           columns = ['DEF_60_CNT_SOCIAL_CIRCLE_0', 'DEF_60_CNT_SOCIAL_CIRCLE_1', 'DEF_60_CNT_SOCIAL_CIRCLE_2',
                                                      'DEF_60_CNT_SOCIAL_CIRCLE_3', 'DEF_60_CNT_SOCIAL_CIRCLE_4', 'DEF_60_CNT_SOCIAL_CIRCLE_5',
                                                      'DEF_60_CNT_SOCIAL_CIRCLE_6'], index=trainset.index)
trainset = pd.concat([trainset, trainset_DEF_60_CNT_SOCIAL_CIRCLE], axis = 1)
del(trainset['DEF_60_CNT_SOCIAL_CIRCLE'])

In [53]:
trainset['DAYS_LAST_PHONE_CHANGE'].mode()

0    0.0
dtype: float64

In [54]:
trainset['DAYS_LAST_PHONE_CHANGE'] = trainset['DAYS_LAST_PHONE_CHANGE'].fillna(0.0)

In [55]:
trainset['DAYS_LAST_PHONE_CHANGE'].unique()

array([ -815., -1562., -1070., ..., -3502., -3788., -3675.])

In [56]:
trainset['DAYS_LAST_PHONE_CHANGE'].isnull().sum()

0

In [57]:
trainset['DAYS_LAST_PHONE_CHANGE'] = scale(trainset['DAYS_LAST_PHONE_CHANGE'])

## 97 ~ 116
- FLAG_DOCUMENT_2
- FLAG_DOCUMENT_3
- FLAG_DOCUMENT_4
- FLAG_DOCUMENT_5
- FLAG_DOCUMENT_6
- FLAG_DOCUMENT_7
- FLAG_DOCUMENT_8
- FLAG_DOCUMENT_9
- FLAG_DOCUMENT_10
- FLAG_DOCUMENT_11
- FLAG_DOCUMENT_12
- FLAG_DOCUMENT_13
- FLAG_DOCUMENT_14
- FLAG_DOCUMENT_15
- FLAG_DOCUMENT_16
- FLAG_DOCUMENT_17
- FLAG_DOCUMENT_18
- FLAG_DOCUMENT_19
- FLAG_DOCUMENT_20
- FLAG_DOCUMENT_21
    - `0` or `1` = > 이미 카테고리 값이라서, 다른 전처리 필요 없음

### 117 ~ 122 
- 117) AMT_REQ_CREDIT_BUREAU_HOUR
- 118) AMT_REQ_CREDIT_BUREAU_DAY
- 119) AMT_REQ_CREDIT_BUREAU_WEEK
- 120) AMT_REQ_CREDIT_BUREAU_MON
- 121) AMT_REQ_CREDIT_BUREAU_QRT
- 122) AMT_REQ_CREDIT_BUREAU_YEAR

In [58]:
print(trainset['AMT_REQ_CREDIT_BUREAU_HOUR'].mode()[0])
print(trainset['AMT_REQ_CREDIT_BUREAU_DAY'].mode()[0])
print(trainset['AMT_REQ_CREDIT_BUREAU_WEEK'].mode()[0])
print(trainset['AMT_REQ_CREDIT_BUREAU_MON'].mode()[0])
print(trainset['AMT_REQ_CREDIT_BUREAU_QRT'].mode()[0])
print(trainset['AMT_REQ_CREDIT_BUREAU_YEAR'].mode()[0])

0.0
0.0
0.0
0.0
0.0
0.0


In [59]:
print(trainset['AMT_REQ_CREDIT_BUREAU_HOUR'].unique())
print(trainset['AMT_REQ_CREDIT_BUREAU_DAY'].unique())
print(trainset['AMT_REQ_CREDIT_BUREAU_WEEK'].unique())
print(trainset['AMT_REQ_CREDIT_BUREAU_MON'].unique())
print(trainset['AMT_REQ_CREDIT_BUREAU_QRT'].unique())
print(trainset['AMT_REQ_CREDIT_BUREAU_YEAR'].unique())

[ 0. nan  1.  2.  3.]
[ 0. nan  1.  3.  4.  2.  5.  6.  9.  8.]
[ 0. nan  1.  3.  2.  5.  6.  4.  8.  7.]
[ 0.  1. nan  6.  2.  7.  9.  3.  4. 11.  5.  8. 16. 12. 14. 10. 13. 17.
 24. 19. 15. 18.]
[  0.   1.  nan   2.   3.   4.   7.   5.   6.   8. 261.]
[ 0.  2. nan  1.  4.  5.  3.  8.  6.  9.  7. 10. 11. 16. 12. 25. 15. 14.
 13. 17. 19. 18.]


In [60]:
print(trainset['AMT_REQ_CREDIT_BUREAU_HOUR'].isnull().sum())
print(trainset['AMT_REQ_CREDIT_BUREAU_DAY'].isnull().sum())
print(trainset['AMT_REQ_CREDIT_BUREAU_WEEK'].isnull().sum())
print(trainset['AMT_REQ_CREDIT_BUREAU_MON'].isnull().sum())
print(trainset['AMT_REQ_CREDIT_BUREAU_QRT'].isnull().sum())
print(trainset['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum())

24766
24766
24766
24766
24766
24766


In [61]:
trainset['AMT_REQ_CREDIT_BUREAU_HOUR'] = trainset['AMT_REQ_CREDIT_BUREAU_HOUR'].fillna(0.0)
trainset['AMT_REQ_CREDIT_BUREAU_DAY'] = trainset['AMT_REQ_CREDIT_BUREAU_DAY'].fillna(0.0)
trainset['AMT_REQ_CREDIT_BUREAU_WEEK'] = trainset['AMT_REQ_CREDIT_BUREAU_WEEK'].fillna(0.0)
trainset['AMT_REQ_CREDIT_BUREAU_MON'] = trainset['AMT_REQ_CREDIT_BUREAU_MON'].fillna(0.0)
trainset['AMT_REQ_CREDIT_BUREAU_QRT'] = trainset['AMT_REQ_CREDIT_BUREAU_QRT'].fillna(0.0)
trainset['AMT_REQ_CREDIT_BUREAU_YEAR'] = trainset['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(0.0)

In [62]:
trainset['AMT_REQ_CREDIT_BUREAU_HOUR'].unique()

array([0., 1., 2., 3.])

In [63]:
from sklearn.preprocessing import LabelBinarizer

trainset_AMT_REQ_CREDIT_BUREAU_HOUR = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['AMT_REQ_CREDIT_BUREAU_HOUR']), 
                                           columns = ['AMT_REQ_CREDIT_BUREAU_HOUR_0', 'AMT_REQ_CREDIT_BUREAU_HOUR_1', 'AMT_REQ_CREDIT_BUREAU_HOUR_2',
                                                      'AMT_REQ_CREDIT_BUREAU_HOUR_3'], index=trainset.index)
trainset = pd.concat([trainset, trainset_AMT_REQ_CREDIT_BUREAU_HOUR], axis = 1)
del(trainset['AMT_REQ_CREDIT_BUREAU_HOUR'])

In [64]:
trainset['AMT_REQ_CREDIT_BUREAU_DAY'].unique()

array([0., 1., 3., 4., 2., 5., 6., 9., 8.])

In [65]:
from sklearn.preprocessing import LabelBinarizer

trainset_AMT_REQ_CREDIT_BUREAU_DAY = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['AMT_REQ_CREDIT_BUREAU_DAY']), 
                                           columns = ['AMT_REQ_CREDIT_BUREAU_DAY_0', 'AMT_REQ_CREDIT_BUREAU_DAY_1', 'AMT_REQ_CREDIT_BUREAU_DAY_2',
                                                      'AMT_REQ_CREDIT_BUREAU_DAY_3', 'AMT_REQ_CREDIT_BUREAU_DAY_4', 'AMT_REQ_CREDIT_BUREAU_DAY_5',
                                                      'AMT_REQ_CREDIT_BUREAU_DAY_6', 'AMT_REQ_CREDIT_BUREAU_DAY_8', 'AMT_REQ_CREDIT_BUREAU_DAY_9'], index=trainset.index)
trainset = pd.concat([trainset, trainset_AMT_REQ_CREDIT_BUREAU_DAY], axis = 1)
del(trainset['AMT_REQ_CREDIT_BUREAU_DAY'])

In [66]:
sorted(trainset['AMT_REQ_CREDIT_BUREAU_WEEK'].unique())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [67]:
from sklearn.preprocessing import LabelBinarizer

trainset_AMT_REQ_CREDIT_BUREAU_WEEK = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['AMT_REQ_CREDIT_BUREAU_WEEK']), 
                                           columns = ['AMT_REQ_CREDIT_BUREAU_WEEK_0', 'AMT_REQ_CREDIT_BUREAU_WEEK_1', 'AMT_REQ_CREDIT_BUREAU_WEEK_2',
                                                      'AMT_REQ_CREDIT_BUREAU_WEEK_3', 'AMT_REQ_CREDIT_BUREAU_WEEK_4', 'AMT_REQ_CREDIT_BUREAU_WEEK_5',
                                                      'AMT_REQ_CREDIT_BUREAU_WEEK_6', 'AMT_REQ_CREDIT_BUREAU_WEEK_7', 'AMT_REQ_CREDIT_BUREAU_WEEK_8'], index=trainset.index)
trainset = pd.concat([trainset, trainset_AMT_REQ_CREDIT_BUREAU_WEEK], axis = 1)
del(trainset['AMT_REQ_CREDIT_BUREAU_WEEK'])

In [68]:
print(sorted(trainset['AMT_REQ_CREDIT_BUREAU_MON'].unique()))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 24.0]


In [69]:
from sklearn.preprocessing import LabelBinarizer

trainset_AMT_REQ_CREDIT_BUREAU_MON = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['AMT_REQ_CREDIT_BUREAU_MON']), 
                                           columns = ['AMT_REQ_CREDIT_BUREAU_MON_0', 'AMT_REQ_CREDIT_BUREAU_MON_1', 'AMT_REQ_CREDIT_BUREAU_MON_2',
                                                      'AMT_REQ_CREDIT_BUREAU_MON_3', 'AMT_REQ_CREDIT_BUREAU_MON_4', 'AMT_REQ_CREDIT_BUREAU_MON_5',
                                                      'AMT_REQ_CREDIT_BUREAU_MON_6', 'AMT_REQ_CREDIT_BUREAU_MON_7', 'AMT_REQ_CREDIT_BUREAU_MON_8',
                                                      'AMT_REQ_CREDIT_BUREAU_MON_9', 'AMT_REQ_CREDIT_BUREAU_MON_10', 'AMT_REQ_CREDIT_BUREAU_MON_11', 
                                                      'AMT_REQ_CREDIT_BUREAU_MON_12', 'AMT_REQ_CREDIT_BUREAU_MON_13', 'AMT_REQ_CREDIT_BUREAU_MON_14',
                                                      'AMT_REQ_CREDIT_BUREAU_MON_15', 'AMT_REQ_CREDIT_BUREAU_MON_16', 'AMT_REQ_CREDIT_BUREAU_MON_17',
                                                      'AMT_REQ_CREDIT_BUREAU_MON_18', 'AMT_REQ_CREDIT_BUREAU_MON_19', 'AMT_REQ_CREDIT_BUREAU_MON_24'], index=trainset.index)
trainset = pd.concat([trainset, trainset_AMT_REQ_CREDIT_BUREAU_MON], axis = 1)
del(trainset['AMT_REQ_CREDIT_BUREAU_MON'])

In [70]:
sorted(trainset['AMT_REQ_CREDIT_BUREAU_QRT'].unique())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 261.0]

In [71]:
from sklearn.preprocessing import LabelBinarizer

trainset_AMT_REQ_CREDIT_BUREAU_QRT = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['AMT_REQ_CREDIT_BUREAU_QRT']), 
                                           columns = ['AMT_REQ_CREDIT_BUREAU_QRT_0', 'AMT_REQ_CREDIT_BUREAU_QRT_1', 'AMT_REQ_CREDIT_BUREAU_QRT_2',
                                                      'AMT_REQ_CREDIT_BUREAU_QRT_3', 'AMT_REQ_CREDIT_BUREAU_QRT_4', 'AMT_REQ_CREDIT_BUREAU_QRT_5',
                                                      'AMT_REQ_CREDIT_BUREAU_QRT_6', 'AMT_REQ_CREDIT_BUREAU_QRT_7', 'AMT_REQ_CREDIT_BUREAU_QRT_8',
                                                      'AMT_REQ_CREDIT_BUREAU_QRT_261'
                                                     ], index=trainset.index)
trainset = pd.concat([trainset, trainset_AMT_REQ_CREDIT_BUREAU_QRT], axis = 1)
del(trainset['AMT_REQ_CREDIT_BUREAU_QRT'])

In [72]:
print(sorted(trainset['AMT_REQ_CREDIT_BUREAU_YEAR'].unique()))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 25.0]


In [73]:
from sklearn.preprocessing import LabelBinarizer

trainset_AMT_REQ_CREDIT_BUREAU_YEAR = pd.DataFrame(LabelBinarizer().
                                                fit_transform(trainset['AMT_REQ_CREDIT_BUREAU_YEAR']), 
                                           columns = ['AMT_REQ_CREDIT_BUREAU_YEAR_0', 'AMT_REQ_CREDIT_BUREAU_YEAR_1', 'AMT_REQ_CREDIT_BUREAU_YEAR_2',
                                                      'AMT_REQ_CREDIT_BUREAU_YEAR_3', 'AMT_REQ_CREDIT_BUREAU_YEAR_4', 'AMT_REQ_CREDIT_BUREAU_YEAR_5',
                                                      'AMT_REQ_CREDIT_BUREAU_YEAR_6', 'AMT_REQ_CREDIT_BUREAU_YEAR_7', 'AMT_REQ_CREDIT_BUREAU_YEAR_8',
                                                      'AMT_REQ_CREDIT_BUREAU_YEAR_9', 'AMT_REQ_CREDIT_BUREAU_YEAR_10', 'AMT_REQ_CREDIT_BUREAU_YEAR_11', 
                                                      'AMT_REQ_CREDIT_BUREAU_YEAR_12', 'AMT_REQ_CREDIT_BUREAU_YEAR_13', 'AMT_REQ_CREDIT_BUREAU_YEAR_14',
                                                      'AMT_REQ_CREDIT_BUREAU_YEAR_15', 'AMT_REQ_CREDIT_BUREAU_YEAR_16', 'AMT_REQ_CREDIT_BUREAU_YEAR_17',
                                                      'AMT_REQ_CREDIT_BUREAU_YEAR_18', 'AMT_REQ_CREDIT_BUREAU_YEAR_19', 'AMT_REQ_CREDIT_BUREAU_YEAR_25'], index=trainset.index)
trainset = pd.concat([trainset, trainset_AMT_REQ_CREDIT_BUREAU_YEAR], axis = 1)
del(trainset['AMT_REQ_CREDIT_BUREAU_YEAR'])

## FINAL FRAME

In [74]:
trainset.shape

(184506, 476)

In [84]:
for i in trainset.columns:
    print(i)

SK_ID_CURR
TARGET
NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MODE
NONLIVINGAREA_MODE
APARTMENTS_MEDI
BA

In [85]:
trainset.to_csv('trainset.csv')